In [1]:
import os
import shutil
import random

dataset_dir = "/Users/ammaster10/Documents/Github/Year4/QMUL/Dataset"
output_dir = "/Users/ammaster10/Documents/Github/Year4/QMUL/DatasetSplit"
splits = ["train", "val", "test"]

# Ratios
train_ratio, val_ratio, test_ratio = 0.7, 0.15, 0.15

# Create output dirs
for split in splits:
    for class_name in ["Normal", "Abnormal", "STE-C", "STE-Mimic", "STE-U"]:
        os.makedirs(os.path.join(output_dir, split, class_name), exist_ok=True)

# Iterate classes
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.isdir(class_dir):
        continue

    if class_name == "STE-C":
        # Collect files from subfolders STE-C-LAD, STE-C-LCX, STE-C-RCA
        files = []
        for sub in os.listdir(class_dir):
            sub_dir = os.path.join(class_dir, sub)
            if os.path.isdir(sub_dir):
                files.extend([os.path.join(sub_dir, f) for f in os.listdir(sub_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    else:
        files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    random.shuffle(files)
    n_total = len(files)
    n_train = int(train_ratio * n_total)
    n_val   = int(val_ratio * n_total)

    train_files = files[:n_train]
    val_files   = files[n_train:n_train+n_val]
    test_files  = files[n_train+n_val:]

    for f in train_files:
        shutil.copy(f, os.path.join(output_dir, "train", class_name, os.path.basename(f)))
    for f in val_files:
        shutil.copy(f, os.path.join(output_dir, "val", class_name, os.path.basename(f)))
    for f in test_files:
        shutil.copy(f, os.path.join(output_dir, "test", class_name, os.path.basename(f)))

print("✅ Dataset split complete into train/val/test with merged STE-C subfolders.")


✅ Dataset split complete into train/val/test with merged STE-C subfolders.


In [ ]:
import os
import numpy as np
from PIL import Image
from glob import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import classification_report, confusion_matrix
